In [ ]:
demo = gr.Blocks()

with demo:
  gr.Markdown("Este es el segundo demo con Blocks")
  with gr.Tabs():
    with gr.TabItem("Transcribe audio en español"):
      with gr.Row():
        audio = gr.Audio(source="microphone", type="filepath")
        transcripcion = gr.Textbox()
      b1 = gr.Button("Transcribe porfa")

    with gr.TabItem("Análisis de sentimiento en español"):
      with gr.Row():
        texto = gr.Textbox()
        label = gr.Label()
      b2 = gr.Button("Sentimiento porfa")

    with gr.TabItem("Clasificación de imagen"):
      with gr.Row():
        image = gr.Image()
        label2 = gr.Label()
      b3 = gr.Button("Clasifica porfa")

    b1.click(audio_a_text, inputs = audio, outputs=transcripcion)
    b2.click(texto_a_sentimiento, inputs=texto, outputs=label)
    b3.click(classify_imagen, inputs=image, outputs=label2)

  

demo.launch()



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

## Introducción a Gradio

Gradio ofrece dos API diferentes según el nivel de detalle que se busque:

- `gradio.Interface`: API de alto nivel que permite crear demos de ML simplemente proporcionando una lista de entradas y salidas.

- `gradio.Blocks`: API de bajo nivel que permite tener un control total sobre los flujos de datos y el diseño de la aplicación. Se pueden crear aplicaciones muy complejas de varios pasos utilizando Blocks (como si fueran bloques de construcción).

Comenzaremos utilizando `Interface` y al final mostraremos un ejemplo de `Blocks`.

## Instalamos Gradio

In [3]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached gradio-3.24.1-py3-none-any.whl (15.7 MB)
  Using cached mdit_py_plugins-0.3.3-py3-none-any.whl (50 kB)
  Using cached gradio_client-0.0.8-py3-none-any.whl (20 kB)
  Using cached python_multipart-0.0.6-py3-none-any.whl (45 kB)
  Using cached aiofiles-23.1.0-py3-none-any.whl (14 kB)
  Using cached huggingface_hub-0.13.4-py3-none-any.whl (200 kB)
  Using cached uvicorn-0.21.1-py3-none-any.whl (57 kB)
  Using cached ffmpy-0.3.0.tar.gz (4.8 kB)
  Preparing metadata (setup.py) ... done
  Using cached httpx-0.23.3-py3-none-any.whl (71 kB)
  Using cached fastapi-0.95.0-py3-none-any.whl (57 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.5/140.5 kB 3.6 MB/s eta 0:00:00
  Using cached websockets-11.0.1-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (129 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
  Using cached se

Ejemplo usando una función para saludar que tiene `text` como input y `text` como output.

In [4]:
import gradio as gr

def saluda(nombre):
  return "hola " + nombre

demo = gr.Interface(fn=saluda, inputs="text", outputs="text")

demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

La clase de `gr.Interface` es una forma fácil de crear demos que pueden ser desde una calculadora hasta una aplicación para reconocimiento de voz. 

Se inicializa con tres parámetros necesarios:


*   `fn`: la función.

*   `inputs`: qué componente(s) usar para los inputs de la función, por ejemplo, "texto", "imagen" o "audio"
* `outputs`: qué componente(s) usar para los outputs de la función, por ejemplo, "texto", "imagen" o "etiqueta"


Gradio incluye más de 20 componentes diferentes, la mayoría de los cuales se pueden utilizar como inputs y outputs. En la documentación está la [lista completa](https://gradio.app/docs/#components).

Ejemplo 2:

In [5]:
import gradio as gr

def saluda(nombre):
  return "hola " + nombre

demo = gr.Interface(fn=saluda, inputs=gr.Textbox(lines=10, placeholder="Dime tu nombre porfa"), outputs="text")

demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

## Demos para clasificación de imágenes

**Ejemplo 1:** demo con modelo cargado de las aplicaciones de TensorFlow.

In [6]:
import tensorflow as tf

inception_net = tf.keras.applications.MobileNetV2()

14536120/14536120 [==============================] - 0s 0us/step


In [29]:
import requests

# Obteniendo las labels de "https://git.io/JJkYN"
respuesta = requests.get("https://git.io/JJkYN")
etiquetas =respuesta.text.split("\n")

def classify_imagen(inp):
  inp = inp.reshape((-1, 224, 224, 3))
  inp = tf.keras.applications.mobilenet_v2.preprocess_input(inp)
  # Modificacion en reshape, para que funcione con decode_predictions
  prediction = inception_net.predict(inp).reshape(1,1000)
  # Nuevo metodo para etiquetar predicciones
  pred_labels = tf.keras.applications.mobilenet_v2.decode_predictions(prediction, top=100)
  confidence = {f'{pred_labels[0][i][1]}': float(pred_labels[0][i][2]) for i in range(100)}
  
  return confidence

In [ ]:
!pip install gradio

In [30]:
import gradio as gr

demo = gr.Interface(fn = classify_imagen,
                    inputs=gr.Image(shape=(224,224)),
                    outputs=gr.Label(num_top_classes=3)
                    )

demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

**Ejemplo 2**: demo cargando un modelo del Hub de Hugging Face. 

In [26]:
import gradio as gr

titulo = "Mi primer demo con Hugging Face"
descripcion = "Este es un demo ejecutado durante la clase con Platzi."

gr.Interface.load(
    "huggingface/google/vit-base-patch16-224",
    inputs=gr.Image(label="Carga una imagen aquí"),
    title = titulo,
    description = descripcion
).launch()

/usr/local/lib/python3.9/dist-packages/gradio/interface.py:93: UserWarning: gr.Intrerface.load() will be deprecated. Use gr.load() instead.
  warnings.warn("gr.Intrerface.load() will be deprecated. Use gr.load() instead.")


Fetching model from: https://huggingface.co/google/vit-base-patch16-224
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

## Demo de transcripción de audio a texto

Usamos `pipeline` para cargar un modelo para automatic speech recognition en español del Hub de Hugging Face.

El demo transcribe automáticamente de lo que grabamos.

In [11]:
%%capture
!pip install transformers
!pip install torch

In [12]:
from transformers import pipeline
import gradio as gr

modelo = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-large-xlsr-53-spanish")

def transcribe(audio):
  text = modelo(audio)["text"]
  return text

In [13]:
gr.Interface(
    fn=transcribe,
    inputs=[gr.Audio(source="microphone", type="filepath")],
    outputs=["textbox"]
).launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

## Blocks

Creamos un demo que recibe dos modelos. Puede transcribir una voz y también puede clasificar el sentimiento de un texto. 

In [14]:
%%capture
!pip install gradio
!pip install transformers
!pip install torch

In [31]:
import gradio as gr
from transformers import pipeline

trans = pipeline("automatic-speech-recognition", model = "facebook/wav2vec2-large-xlsr-53-spanish")
clasificador = pipeline("text-classification", model = "pysentimiento/robertuito-sentiment-analysis")

In [32]:
def audio_a_text(audio):
  text = trans(audio)["text"]
  return text

def texto_a_sentimiento(text):
  return clasificador(text)[0]["label"]

In [33]:
demo = gr.Blocks()

with demo:
  gr.Markdown("Demo para la clase de Platzi")
  audio = gr.Audio(source="microphone", type="filepath")
  texto = gr.Textbox()
  b1 = gr.Button("Transcribe porfa")
  b1.click(audio_a_text, inputs=audio, outputs=texto)

  label = gr.Label()
  b2 = gr.Button("Clasifica porfa el sentimiento")
  b2.click(texto_a_sentimiento, inputs=texto, outputs=label)

demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Hagamos un aplicación de blocks un poco más interesante.

In [34]:
demo = gr.Blocks()

with demo:
  gr.Markdown("Este es el segundo demo con Blocks")
  with gr.Tabs():
    with gr.TabItem("Transcribe audio en español"):
      with gr.Row():
        audio = gr.Audio(source="microphone", type="filepath")
        transcripcion = gr.Textbox()
      b1 = gr.Button("Transcribe porfa")

    with gr.TabItem("Análisis de sentimiento en español"):
      with gr.Row():
        texto = gr.Textbox()
        label = gr.Label()
      b2 = gr.Button("Sentimiento porfa")

    with gr.TabItem("Clasificación de imagen"):
      with gr.Row():
        image = gr.Image(shape=(224,224))
        label2 = gr.Label(num_top_classes=3)
      b3 = gr.Button("Clasifica porfa")

    b1.click(audio_a_text, inputs = audio, outputs=transcripcion)
    b2.click(texto_a_sentimiento, inputs=texto, outputs=label)
    b3.click(classify_imagen, inputs=image, outputs=label2)
 

demo.launch()



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>